In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15339, done.
remote: Total 15339 (delta 0), reused 0 (delta 0), pack-reused 15339
Receiving objects: 100% (15339/15339), 14.00 MiB | 16.51 MiB/s, done.
Resolving deltas: 100% (10312/10312), done.


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9049104022823931678, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16185556992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7747554190625187559
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

##### -- Makefile 수정 
GPU=1    
CUDNN=1    
CUDNN_HALF=1    
OPENCV=1    
AVX=0    
OPENMP=0    
LIBSO=1    
ZED_CAMERA=0    
ZED_CAMERA_v2_8=0        

##### -- 아래 URL을 참고하여 Colab에서 사용하고 있는 GPU에 맞는 주석 해제 
##### SM60 or SM_60, compute_60  -> 주석 해제
http://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/


In [ ]:
%cd darknet/
!make
!mkdir Swoosh_Detection
%cd ../

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if 

In [ ]:
!cp /content/gdrive/MyDrive/annotation.zip /content/darknet/Swoosh_Detection/annotation.zip

In [ ]:
!unzip -P "패스워드" /content/darknet/Swoosh_Detection/annotation.zip -d /content/darknet/Swoosh_Detection/annotation/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 N762.png  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 N762.xml  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 N765.png  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 N765.xml  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 PREMIUM100.png  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 PREMIUM100.xml  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 PREMIUM11.png  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 PREMIUM11.xml  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 PREMIUM13.png  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 PREMIUM13.xml  
  inflating: /content/darknet/Swoosh_Detection/annotation/AIR MAX 90 PREMIUM14.png  
  inflating: /content/darknet/Swoosh_Detection/

In [ ]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

%cd /content
dirs = ['darknet/Swoosh_Detection/annotation/']
classes = ['H/F_Welding','Stitching']


def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.png'):
        image_list.append(filename)
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)

    return image_list


def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)



def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml' ,encoding='UTF8')
    out_file = open(output_path + basename_no_ext + '.txt', 'w' ,encoding='UTF8')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

cwd = getcwd()

for dir_path in dirs:
    full_dir_path = cwd + '/' + dir_path
    output_path = full_dir_path # + '/yolo/'
    

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    print(full_dir_path)
    list_file = open(full_dir_path + 'annotation' + '.txt', 'w')

    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)
        
    list_file.close()

    print("Finished processing: " + dir_path)
    print(full_dir_path)

/content
/content/darknet/Swoosh_Detection/annotation/
Finished processing: darknet/Swoosh_Detection/annotation/
/content/darknet/Swoosh_Detection/annotation/


In [ ]:
import random 
txt = open('/content/darknet/Swoosh_Detection/annotation/annotation.txt','r') 
f = open('/content/darknet/Swoosh_Detection/annotation/annotation_shuffle.txt','w') 
tmp = [] 
while True : 
  line = txt.readline() 
  if not line: 
    break 
  tmp.append(line) 
random.shuffle(tmp) 
for i in tmp : 
  f.write(i) 
txt.close() 
f.close()

In [ ]:
!mkdir /content/darknet/custom/

In [ ]:
count = 0 
length = 10499 #total line 
txt = open('/content/darknet/Swoosh_Detection/annotation/annotation_shuffle.txt','r') 
i = 0 
f = open('/content/darknet/custom/train.txt','w') 
f2 = open('/content/darknet/custom/valid.txt','w') 
while True : 
  if i == 0 : 
    line = txt.readline() 
    if not line : 
      break 
    count +=1 
    if count < int(length/10)*2 : 
      f2.write(line) 
    else : 
      f.write(line) 
txt.close() 
f.close() 
f2.close()

In [ ]:
!gdown --id 1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp -O /content/darknet/yolov4.conv.137

Downloading...
From: https://drive.google.com/uc?id=1JKF-bdIklxOOVy-2Cr5qdvjgGpmGfcbp
To: /content/darknet/yolov4.conv.137
100% 170M/170M [00:01<00:00, 146MB/s]


### 1. darknet 디렉토리로 이동하여 cfg파일을 수정
subdivisions = 16 으로 수정
max_batches는 본인 데이터의 클래스 갯수 * 2000 을 적어주면 된다. 
필자의 클래스는 15개이므로 15*2000 = 30000 을 입력
steps는 max_batches의 80%의 수치와 90%의 수치를 작성

- yolo 부분에 있는 classes와 yolo위쪽에 있는 filter의 크기를 수정
classes는 훈련시키고자 하는 데이터셋의 class갯수, filter의 갯수는 (classes+5) * 3


### 2. custom.names 파일 생성
- Class이름을 index순서대로 쭉 작성

### 3. custom.data 파일 생성
- 클래스의 갯수, train set 경로, validation set경로, names파일의 경로, weight파일이 출력될 경로를 입력

classes= 1     
train = custom/train.txt      
valid = custom/validation.txt      
names = custom/custom.names      
backup = backup/     


In [ ]:
%cd /content/darknet

/content/darknet


In [ ]:
!./darknet detector train /content/darknet/custom/custom.data /content/darknet/cfg/yolov4.cfg /content/darknet/yolov4.conv.137 -map -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 total_bbox = 1680502, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.887870), count: 5, class_loss = 0.230369, iou_loss = 1.893598, total_loss = 2.123968 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.893554), count: 10, class_loss = 0.102441, iou_loss = 1.078685, total_loss = 1.181126 
 total_bbox = 1680517, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.751998), count: 1, class_loss = 0.172649, iou_loss = 1.386580, total_loss = 1.559229 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.806279), count: 7, class_loss = 0.185052, iou_loss = 2.672649, total_loss = 2.857701 
v3 (iou loss, 

In [ ]:
from google.colab import files
files.download('/content/darknet/backup/yolov4_1000.weights')
files.download('/content/darknet/backup/yolov4_2000.weights')
files.download('/content/darknet/backup/yolov4_best.weights')
files.download('/content/darknet/backup/yolov4_final.weights')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>